Querying/filtering of the Stats Can building permits data

full table downloaded from here:
https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3410028501

over 7gb or so unzipped

In [1]:
import pandas as pd

In [2]:
filename = "input-data/34100285.csv"

df = []

chunksize = 10 ** 6
with pd.read_csv(filename, chunksize=chunksize) as reader:
    for chunk in reader:
        filtered_chunk = chunk[chunk["Seasonal adjustment, value type"] == "Unadjusted, current"]
        filtered_chunk = filtered_chunk[
            filtered_chunk["Variables"] == "Number of dwelling-units created"
        ]
        filtered_chunk = filtered_chunk[
            filtered_chunk["Type of building"] == "Total residential and non-residential"
        ]
        filtered_chunk = filtered_chunk[
            (filtered_chunk["Type of work"] == "New construction") | 
            (filtered_chunk["Type of work"] == "Conversion from single to multiple dwelling") | 
            (filtered_chunk["Type of work"] == "Conversion from multiple to multiple dwelling") | 
            (filtered_chunk["Type of work"] == "Conversion from non-residential to residential building")
        ]
        if len(filtered_chunk) > 0:
            df.append(filtered_chunk)

In [3]:
df = pd.concat(df, ignore_index=True)

In [4]:
df = df[["REF_DATE","GEO", "Type of work","VALUE"]]

In [5]:
# df[df["GEO"] == "Toronto, Ontario"].to_csv("input-data/toronto-subset.csv")

In [6]:
df['Type'] = df['Type of work'].replace(
    {
        'New construction': 'new',
        'Conversion from single to multiple dwelling': 's-to-m',
        'Conversion from multiple to multiple dwelling': 'm-to-m',
        'Conversion from non-residential to residential building': 'n-to-r'
    }
)

In [7]:
df.to_csv("conversions-canada-2018-to-2024.csv", index=False)

In [8]:
df

,REF_DATE,GEO,Type of work,VALUE,Type
0,2018-01,Canada,New construction,13498.0,new
1,2018-01,Canada,Conversion from single to multiple dwelling,475.0,s-to-m
2,2018-01,Canada,Conversion from multiple to multiple dwelling,384.0,m-to-m
3,2018-01,Canada,Conversion from non-residential to residential...,143.0,n-to-r
4,2018-01,Newfoundland and Labrador,New construction,23.0,new
...,...,...,...,...,...
20059,2024-04,"Vancouver, British Columbia",Conversion from non-residential to residential...,4.0,n-to-r
20060,2024-04,"Victoria, British Columbia",New construction,727.0,new
20061,2024-04,"Victoria, British Columbia",Conversion from single to multiple dwelling,16.0,s-to-m
20062,2024-04,"Victoria, British Columbia",Conversion from multiple to multiple dwelling,0.0,m-to-m
